In [ ]:
import pandas as pd
import numpy as np
import networkx as nx


# 새 섹션

In [ ]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [ ]:
import matplotlib.pyplot as plt

#matplotlib.rc("font",family ="NanumGothic" )

In [ ]:
plt.rc("font",family ="NanumGothic" )

In [ ]:
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df_raw = pd.read_csv('/content/drive/MyDrive/total_data.csv의 사본',encoding ='utf-8')

In [ ]:
df_raw.reset_index(drop=False,inplace=True)
df_raw

In [ ]:
cond1 = df_raw['카테고리'].isin(['의류'])
df_raw1 = df_raw.loc[cond1]
df_p1 = df_raw1.pivot_table(index='PD', values='목표달성여부',aggfunc=['sum','count']).reset_index()
df_p1.columns = df_p1.columns.droplevel(1)
p = df_p1['ratio'] = df_p1['sum'] / df_p1['count']
idx = df_p1[df_p1['count']<20].index
df_p1.drop(idx,inplace=True)


In [ ]:
cond1 = df_raw['카테고리'].isin(['의류'])
df_raw1 = df_raw.loc[cond1]

df_p2 = df_raw1.pivot_table(index='SH', values='목표달성여부',aggfunc=['sum','count']).reset_index()
df_p2.columns = df_p2.columns.droplevel(1)
p = df_p2['ratio'] = df_p2['sum'] / df_p2['count']
idx = df_p2[df_p2['count']<20].index
df_p2.drop(idx,inplace=True)
df_p2

In [ ]:
df_p3 = df_raw.pivot_table(index='MD담당자명', values='목표달성여부',aggfunc=['sum','count']).reset_index()
df_p3.columns = df_p3.columns.droplevel(1)
p = df_p3['ratio'] = df_p3['sum'] / df_p3['count']
idx = df_p3[df_p3['count']<20].index
df_p3.drop(idx,inplace=True)

In [ ]:
a = df_p1.sort_values(by=['ratio'], ascending=False)
t = a.reset_index()
t

In [ ]:
b = df_p2.sort_values(by=['ratio'], ascending=False)
B = b['SH'].reset_index()
SH = B['SH'][0]
B

In [ ]:
c = df_p3.sort_values(by=['ratio'], ascending=False)
C = c['MD담당자명'].reset_index()
MD = ['MD담당자명'][0]

# 생방송만 체크

In [ ]:
df_raw.카테고리

In [ ]:
df_raw.PD= df_raw.PD.str.split(',').str[0]

In [ ]:
df_raw.PD.replace(np.nan, '', regex=True,inplace=True)

In [ ]:
df_raw.PD

In [ ]:
df_raw.SH= df_raw.SH.str.split(',').str[0]

In [ ]:
df_raw.SH.replace(np.nan, '', regex=True,inplace=True)

In [ ]:
df_raw.TD= df_raw.TD.str.split(',').str[0]

In [ ]:
df_raw.TD.replace(np.nan, '', regex=True,inplace=True)

In [ ]:
df_raw.MD담당자명= df_raw.MD담당자명.str.split(',').str[0]
df_raw.MD담당자명

In [ ]:
df_raw.MD담당자명.replace(np.nan, '', regex=True,inplace=True)

In [ ]:
df_raw['목표달성여부'] =df_raw['상품순주문금액']-df_raw['상품목표주문금액']

In [ ]:
# 목표변수 안될시 이 함수와 아래 apply 코드를 쓰세용
def conversion(x):
  if x>=0 : x=1
  else : x=0
  return x

In [ ]:
df_raw['목표달성여부']=df_raw['목표달성여부'].apply(lambda x : conversion(x))

In [ ]:
t=[]

for i in range(len(df_raw.PD)):
    if df_raw.목표달성여부[i] == 1:
        df_raw.목표달성여부[i] ='성공'
        t.append([df_raw.PD[i],df_raw.SH[i],df_raw.TD[i],df_raw.MD담당자명[i]])
print(type(t))    

In [ ]:
encoding = pd.DataFrame(t)

In [ ]:
encoding

In [ ]:
cate = list(set(df_raw['카테고리']))
cate

In [ ]:
cond1 = (df_raw['카테고리'].isin(['신선수산']))

df_raw_cate = df_raw.loc[cond1]
df_raw_cate = df_raw_cate.drop(index=df_raw_cate.loc[df_raw_cate.목표달성여부==0].index)
df_raw_cate.drop('index',axis=1,inplace=True)
df_raw_cate.reset_index(drop=True,inplace=True)

In [ ]:
t = []
for i in range(len(df_raw_cate)):
    df_raw_cate['목표달성여부'] ='성공'
    t.append([df_raw_cate['PD'][i],df_raw_cate['SH'][i],df_raw_cate['MD담당자명'][i]])

In [ ]:
enc = TransactionEncoder()
df_raw_enc=enc.fit_transform(X=t)
df_asso = pd.DataFrame(df_raw_enc,columns=enc.columns_)
df_asso.head()

In [ ]:
#df_raw.to_excel('/home/piai/바탕화면/2. Big Data 분석/df_raw4.xlsx')

In [ ]:
v_min_support=0.001
df_freq = apriori(df_asso,min_support=v_min_support,use_colnames=True)
df_asso_rule_신선수산 = association_rules(df_freq,metric = 'confidence',min_threshold=0.001)

In [ ]:
cond1 = (df_asso_rule_신선수산['support'] > 0.01)
df_asso_rule_신선수산.loc[cond1].sort_values(by=['support','lift'], ascending=False)
cond2 = (df_asso_rule_신선수산['lift'] > 1 )
b =df_asso_rule_신선수산.loc[cond2].sort_values(by=['support','lift'], ascending=False)

In [ ]:
#df_raw.to_excel('/home/piai/바탕화면/2. Big Data 분석/df_raw4.xlsx')

In [ ]:
cond1 = (df_asso_rule_신선수산['support'] > 0.01)
df_asso_rule_신선수산.loc[cond1].sort_values(by=['support','lift'], ascending=False)
cond2 = (df_asso_rule_신선수산['lift'] > 1 )
b =df_asso_rule_신선수산.loc[cond2].sort_values(by=['support','lift'], ascending=False)
t = [i for i in b['antecedents']]
k = [i for i in frozenset(t[0])]
A = [i for i in b['consequents']]
B = [i for i in frozenset(A[0])]
print(k,B)
b.head(5).to_excel('신선수산_연관분석1.xlsx')

In [ ]:
t=['PD','SH','MD']
if len(df_raw[df_raw['PD']==k[0]]) ==0:
  if len(df_raw[df_raw['SH']==k[0]]) == 0:
    t[2] = k[0]
  else:
    t[1] = k[0]
else:
  t[0] = k[0]
if len(df_raw[df_raw['PD']==B[0]]) ==0:
  if len(df_raw[df_raw['SH']==B[0]]) == 0:
    t[2] = B[0]
  else:
    t[1] = B[0]
else:
  t[0] = B[0]
for i in t:
  if '1' in t:
    t[0] = 'PD'
  elif '2' in t:
    t[1] = 'SH'
  elif '3' in t:
    t[2] = 'MD'
print(t)

In [ ]:


cond1 = (df_raw['카테고리'].isin(['신선수산']))
df_raw1 = df_raw.loc[cond1]
if t[0] == 'PD':
  List= (df_raw1.loc[(df_raw1['SH'] == t[1])&(df_raw1['MD담당자명']==t[2]),])
if t[1] == 'SH':
  List = df_raw1.loc[(df_raw1['PD'] == t[0])& (df_raw1['MD담당자명']==t[2]), ]
if t[2] == 'MD':
  List = df_raw1.loc[(df_raw1['PD'] == t[0])& (df_raw1['SH']==t[1]),]
List[['PD','SH','MD담당자명','매출초과비율']].sort_values(by=['매출초과비율'],ascending=False).head(5).to_excel('신선수산_연관분석2.xlsx')

In [ ]:
# cond1 = (df_raw['카테고리'].isin(['건강식품']))

# df_raw_cate = df_raw.loc[cond1]
# df_raw_cate = df_raw_cate.drop(index=df_raw_cate.loc[df_raw_cate.목표달성여부==0].index)
# df_raw_cate.drop('index',axis=1,inplace=True)
# df_raw_cate.reset_index(drop=True,inplace=True)

In [ ]:
# t = []
# for i in range(len(df_raw_cate)):
#     df_raw_cate['목표달성여부'] ='성공'
#     t.append([df_raw_cate['PD'][i],df_raw_cate['SH'][i],df_raw_cate['MD담당자명'][i]])
# print(t[0])

In [ ]:
# enc = TransactionEncoder()
# df_raw_enc=enc.fit_transform(X=t)
# df_asso = pd.DataFrame(df_raw_enc,columns=enc.columns_)
# df_asso.head()

In [ ]:
# v_min_support=0.001
# df_freq = apriori(df_asso,min_support=v_min_support,use_colnames=True)
# df_asso_rule_건강식품 = association_rules(df_freq,metric = 'confidence',min_threshold=0.001)
# cond1 = (df_asso_rule_건강식품['support'] > 0.01 )
# cond2 = (df_asso_rule_건강식품['lift'] > 1 )
# a = df_asso_rule_건강식품.loc[cond1].sort_values(by=['support','lift'], ascending=False)
# cond3 = (a['lift']>1)
# b = a.loc[cond2].sort_values(by=['support','lift'], ascending=False)
# b.reset_index()

In [ ]:
# cond1 = (df_raw['카테고리'].isin(['인테리어']))
# df_raw_cate = df_raw.loc[cond1]
# df_raw_cate = df_raw_cate.drop(index=df_raw_cate.loc[df_raw_cate.목표달성여부==0].index)
# df_raw_cate.drop('index',axis=1,inplace=True)
# df_raw_cate.reset_index(drop=True,inplace=True)
# t = []
# for i in range(len(df_raw_cate)):
#     df_raw_cate_목표달성여부=1
#     t.append([df_raw_cate['PD'][i],df_raw_cate['SH'][i],df_raw_cate['MD담당자명'][i]])
#     enc = TransactionEncoder()

# df_raw_enc=enc.fit_transform(X=t)
# df_asso = pd.DataFrame(df_raw_enc,columns=enc.columns_)
# df_asso.head()
# v_min_support=0.05
# df_freq = apriori(df_asso,min_support=v_min_support,use_colnames=True)
# df_asso_rule_인테리어 = association_rules(df_freq,metric = 'confidence',min_threshold=0.05)

# cond1 = (df_asso_rule_인테리어['support'] > 0.01 )
# cond2 = (df_asso_rule_인테리어['lift'] > 1 )
# a = df_asso_rule_인테리어.loc[cond1].sort_values(by=['support','lift'], ascending=False)
# cond3 = (a['lift']>1)
# b = a.loc[cond2].sort_values(by=['support','lift'], ascending=False)
# b
# #박상언 - SH, 국성훈 - PD, 전상운 -MD

In [ ]:
# cond1 = (df_raw['카테고리'].isin(['생활용품']))
# df_raw_cate = df_raw.loc[cond1]
# df_raw_cate = df_raw_cate.drop(index=df_raw_cate.loc[df_raw_cate.목표달성여부==0].index)
# df_raw_cate.drop('index',axis=1,inplace=True)
# df_raw_cate.reset_index(drop=True,inplace=True)
# t = []
# for i in range(len(df_raw_cate)):
#     t.append([df_raw_cate['PD'][i],df_raw_cate['SH'][i],df_raw_cate['MD담당자명'][i]])
#     enc = TransactionEncoder()

# df_raw_enc=enc.fit_transform(X=t)
# df_asso = pd.DataFrame(df_raw_enc,columns=enc.columns_)
# df_asso.head()
# v_min_support=0.005
# df_freq = apriori(df_asso,min_support=v_min_support,use_colnames=True)
# df_asso_rule_인테리어 = association_rules(df_freq,metric = 'confidence',min_threshold=0.005)

# cond1 = (df_asso_rule_인테리어['support'] > 0.01 )
# cond2 = (df_asso_rule_인테리어['lift'] > 1 )
# a = df_asso_rule_인테리어.loc[cond1].sort_values(by=['support','lift'], ascending=False)
# cond3 = (a['lift']>1)
# b = a.loc[cond2].sort_values(by=['support','lift'], ascending=False)
# b.reset_index()
# #박상언 - SH, 국성훈 - PD, 전상운 -MD

In [ ]:
# cond1 = (df_raw['카테고리'].isin(['의류']))
# df_raw_cate = df_raw.loc[cond1]
# df_raw_cate = df_raw_cate.drop(index=df_raw_cate.loc[df_raw_cate.목표달성여부==0].index)
# df_raw_cate.drop('index',axis=1,inplace=True)
# df_raw_cate.reset_index(drop=True,inplace=True)
# t = []
# for i in range(len(df_raw_cate)):
#     df_raw_cate.목표달성여부=1
#     t.append([df_raw_cate['PD'][i],df_raw_cate['SH'][i],df_raw_cate['MD담당자명'][i]])
#     enc = TransactionEncoder()

# df_raw_enc=enc.fit_transform(X=t)
# df_asso = pd.DataFrame(df_raw_enc,columns=enc.columns_)
# df_asso.head()
# v_min_support=0.001
# df_freq = apriori(df_asso,min_support=v_min_support,use_colnames=True)
# df_asso_rule_의류 = association_rules(df_freq,metric = 'confidence',min_threshold=0.001)

# cond1 = (df_asso_rule_의류['support'] > 0.01 )
# cond2 = (df_asso_rule_의류['lift'] > 1 )
# a = df_asso_rule_의류.loc[cond1].sort_values(by=['support','lift'], ascending=False)
# cond3 = (a['lift']>1)
# b = a.loc[cond2].sort_values(by=['support','lift'], ascending=False)
# b['antecedents']
# t = [i for i in b['antecedents']]
# k = [i for i in frozenset(t[0])]
# A = [i for i in b['consequents']]
# B = [i for i in frozenset(A[0])]
# b
# #강승필 - pd, 최소유 - sh,  